# Examine Each Individual Interaction
Code to automatically take each event and display it in a reaction formate along with the kinetic energy of each particle involved, wght(s) and reaction (res, mec, dis, qel). If there are more than one of a particle or reaction (incoming or outgoing) the number will be displayed instead of multiple of the same.

In [1]:
## Import statements ##
import numpy as np
import pandas as pd
import time
import awkward as ak
from particle import Particle
import matplotlib.pyplot as plt

In [2]:
## Read in necessary dataframes (fp: final particles, ip: initial particles, sm all other columns) ##
gst_fp_df = pd.read_hdf("/Users/laurazichi/Desktop/Fermilab/base_generation_with_weights_13Jun_FSIFix.hdf", "gst_fp_df")
gst_ip_df = pd.read_hdf("/Users/laurazichi/Desktop/Fermilab/base_generation_with_weights_13Jun_FSIFix.hdf", "gst_ip_df")
gst_df = pd.read_hdf("/Users/laurazichi/Desktop/Fermilab/base_generation_with_weights_13Jun_FSIFix.hdf", "gst_df")

In [3]:
## Find The Relevant weights ##
cols = gst_df_sm.columns[81:113]
wghts_new = []
for wght in cols:
    if(wght.split('_')[3] == 'p1'):
        wghts_new.append(wght)

In [37]:
print(wghts_new)

['wght_FrCEx_N_p1', 'wght_FrAbs_N_p1', 'wght_FrInel_N_p1', 'wght_FrPiProd_N_p1', 'wght_FrCEx_pi_p1', 'wght_FrAbs_pi_p1', 'wght_FrInel_pi_p1', 'wght_FrPiProd_pi_p1']


In [4]:
def add_scattering_label(df):   
    '''For a given dataframe of events add a column to the df ('scatt_lab') that has the specific type of 
    scattering reaction (qel, res, or dis)'''
    
    qel = df['qel']
    qel = np.where(qel == True, 'qel', '')

    res = df['res']
    res = np.where(res == True, 'res', '')

    dis = df['dis']
    dis = np.where(dis == True, 'dis', '')

    label = np.core.defchararray.add(dis, res)
    label = np.core.defchararray.add(label, qel)
    
    df['scatt_lab'] = label
    return df

In [5]:
def par_engy(pars, kes):
    '''For an array of particles and an array of their corresponding energies, return a string formated with 
    each particles symbol and corresponding energy separated by commas'''
    
    str_engy = ''
    
    for par, ke in zip (pars, kes):
        
        p = Particle.from_pdgid(par)
        str_engy = str_engy + p.name + ': ' + str(round(ke, ndigits = 3)) + ', '
        
    return str_engy[:-2]

In [6]:
def find_reaction(pari, parsf):
    '''Return a string of a symbol based reaction between the initial particle and final particle(s)'''
    
    str_parsf = ''
    
    parsf = parsf.tolist()
    
    # Go through unique final particles
    for par in np.unique(parsf):
        p = Particle.from_pdgid(par)
        
        # Find the number of times they appear and append number if not 1
        count = (str(parsf.count(par)) if parsf.count(par) > 1 else '')
        str_parsf = str_parsf + count + p.name + ' + '
        
        
    return Particle.from_pdgid(pari).name + ' -> ' + str_parsf[:-3]    

In [7]:
def find_react_large(parsi, resc, parsf):
    '''Return a string of a symbol representation of the reaction between an array of initial particles and
    final particles. Second string returned are the particles with resc num of 1.'''
    
    str_parsi = ''
    str_parsf = ''
    str_removed = ''
    
    # Remove particles from initial particle array with resc of 1
    removed = parsi[np.where(resc == 1)]
    index = np.where(resc > 1)
    parsi = parsi[index]
    
    parsf = parsf.tolist()
    
    # Remove those particles from the final particle array (symbol based so exact one does not matter)
    # Added removed ones to removed string which is later returned
    for p in removed:
        parsf.remove(p)
        str_removed = str_removed + Particle.from_pdgid(p).name + ', '
        
    # Go through initial particles array and put it into a string of symbols
    for par in parsi:
        p = Particle.from_pdgid(par)
        str_parsi = str_parsi + p.name + ' + '

    
    # Go through final particles array and put it into a string of symbols
    for par in np.unique(parsf):
        p = Particle.from_pdgid(par)
        
        # Account for duplicates and add num if greater than 1
        count = (str(parsf.count(par)) if parsf.count(par) > 1 else '')
        str_parsf = str_parsf + count + p.name + ' + '

    
    return str_parsi[:-3] + '->' + str_parsf[:-3], str_removed[:-2]

In [8]:
def make_reaction(parsi, rescs, parsf):
    '''Take in the full array of initial and final particles along with resc numbers for each event
    and use corresponding reaction functions to find a symbolic representation of them'''
    
    ## Remove Nans from parsf (result from combining initial and final particles arrays)
    parsf = parsf[~(pd.isnull(parsf))]
    parsi = parsi[~np.isnan(parsi)]
    
    ## Resc numbs correspond to individual reactions
    if(np.sum(rescs) == np.shape(parsf)[0]):   
        
        str_removed = ''
        str_reactions = ''
        reactions = []
        
        # Go through and take out reactions based resc, if resc = 1 add to removed string
        for i, (resc, pari) in enumerate (zip (rescs, parsi)):
            if(resc != 1):
                str_reaction = find_reaction(pari, parsf[i:i+int(resc)])
                reactions.append(str_reaction)
                 
            else:
                str_removed = str_removed + Particle.from_pdgid(pari).name + ', '
        
        # Find the number of reactions if duplicated
        for react in np.unique(reactions):
            count = reactions.count(react)
        
            if(count > 1):     
                str_reactions = str_reactions + str(count) + '(' + react + '), ' 
            else:
                str_reactions = str_reactions + react + ', ' 
                
        return str_reactions[:-2], str_removed[:-2]
    
    ## If resc nums do not seem to work, do entire reaction at once
    else: 
        return  find_react_large(parsi, rescs, parsf)
    

In [9]:
def extract_reaction(gst_df, ip_df, fp_df, wght, thresh = 20):
    '''Go through dataframes of wghts, initial particles, and final particles and return a df with a symbolic
    representation of each reaction, pars with resc = 1, keis, kefs, and scattering label for a designated wght.
    All events selected correspond to those with wght greater than 1.'''
    
    # Get index of where that wght is greater than 1
    ind = np.where(gst_df[wght] > 1)[0]
    index = gst_df.iloc[ind].index

    # Get name, rescatering and kinetic energy of particles 
    ip_df = ip_df[['pdgi','resc','kei']].loc[index[:thresh]]
    fp_df = fp_df[['pdgf','kef']].loc[index[:thresh]]

    # Add frames together to pass values to function 
    df_total = pd.concat([ip_df, fp_df], axis = 1, sort = True)
    
    # Find the reaction of every row 
    reaction = df_total.groupby("entry").apply(lambda x: make_reaction(x.pdgi.to_numpy(), 
                                                        np.nan_to_num(x.resc.to_numpy()), x.pdgf.to_numpy()))
    arr = np.hstack(reaction.to_numpy()).reshape(thresh,2)
    
    # Add the rows to the new dataframe to return 
    df_new = pd.DataFrame()
    df_new['reaction'] = arr[::,0]
    df_new['resc = 1'] = arr[::,1]

    # Set the indices by event instead of in order 
    df_new = df_new.set_index(reaction.index)
     
    # Add all relevant weights to the end 
    wghts_df = gst_df[[wght, 'scatt_lab']].loc[index[:thresh]]
    wghts_df[wghts_df == 1] = np.nan
    
    # Find the Kei and Kef of each event 
    kei_arr = ip_df.groupby("entry").apply(lambda x: par_engy(x.pdgi, x.kei))
    kef_arr = fp_df.groupby("entry").apply(lambda x: par_engy(x.pdgf, x.kef))
    
    df_new['kei'] = kei_arr
    df_new['kef'] = kef_arr
    
    
    df_new = pd.concat([df_new, wghts_df], axis = 1)
    
    return df_new

In [8]:
### Perform zero and pt (400 MeV) cut on large dataframe ###
## Remove Zeros ##
gst_fp_df = gst_fp_df[~(gst_fp_df['pyf'] == 0)]

## Pt of ptl > 400 MeV cut ##
gst_df_sm = gst_df_sm[(np.sqrt(gst_df_sm["pxl"]*gst_df_sm["pxl"]+gst_df_sm["pyl"]*gst_df_sm["pyl"]) > 0.4)]

In [18]:
## Add type of reaction label (dis, qel, res)
gst_df_sm = add_scattering_label(gst_df_sm)

In [38]:
# Go through all weights 
for wght in wghts_new:
    df = extract_reaction(gst_df_sm, gst_ip_df, gst_fp_df, wght)
    display(df)

,reaction,resc = 1,kei,kef,wght_FrCEx_N_p1,scatt_lab
10,p -> n + p,pi-,"p: 0.095, pi-: 0.053","n: 0.083, p: 0.03, pi-: 0.053",1.5,dis
59,n -> n + p,,n: 0.165,"p: 0.067, n: 0.024",1.5,qel
62,p + pi- + pi-->pi- + pi0 + pi+ + 3n + p,"pi+, pi+","p: 0.735, pi+: 0.027, pi-: 0.255, pi+: 0.704, ...","n: 0.472, p: 0.251, pi+: 0.027, pi0: 0.214, n:...",1.5,dis
70,p + pi+->pi- + 4n + 4p,,"p: 0.049, pi+: 0.219","pi-: 0.34, n: 0.012, p: 0.045, p: 0.049, p: 0....",1.5,res
114,p->n,,p: 0.161,n: 0.1,1.5,qel
123,p -> n + p,,p: 0.197,"n: 0.006, p: 0.137",1.5,qel
132,p->p,,p: 0.056,p: 0.037,1.5,qel
159,p + pi-->pi- + n + 2p,,"p: 0.351, pi-: 0.116","n: 0.22, p: 0.137, pi-: 0.081, p: 0.025",1.5,dis
163,n->n,,n: 0.084,n: 0.057,1.5,qel
191,p -> n + p,,p: 0.208,"n: 0.133, p: 0.019",1.5,qel


,reaction,resc = 1,kei,kef,wght_FrAbs_N_p1,scatt_lab
29,p->21n + 21p,"pi-, pi0","p: 0.338, pi-: 2.06, pi0: 0.381","p: 0.007, p: 0.004, p: 0.005, p: 0.021, p: 0.0...",1.200000,dis
83,p->4n + p,,p: 0.402,"p: 0.17, n: 0.152, n: 0.005, n: 0.075, n: 0.024",1.200000,qel
162,p + pi-->6n + 6p,,"p: 0.426, pi-: 0.415","p: 0.091, p: 0.062, p: 0.045, p: 0.062, n: 0.1...",1.200000,res
214,n + pi0->5n + 4p,,"n: 0.111, pi0: 0.177","p: 0.023, n: 0.017, n: 0.025, n: 0.066, p: 0.0...",1.200000,res
262,n + p->9n + 5p,,"n: 0.127, p: 0.304","p: 0.001, p: 0.005, p: 0.011, p: 0.004, n: 0.0...",1.004963,
274,p + pi+ + pi-->pi0 + 9n + 7p,,"p: 0.131, pi+: 0.021, pi-: 0.452","p: 0.006, p: 0.065, n: 0.061, n: 0.019, pi0: 0...",1.200000,dis
282,p -> 2n + 2p,,p: 0.091,"p: 0.019, p: 0.012, n: 0.007, n: 0.073",1.200000,qel
307,n->12n + 7p,"pi+, pi0","n: 0.252, pi+: 0.57, pi0: 0.386","p: 0.021, p: 0.015, n: 0.002, n: 0.017, p: 0.0...",1.200000,dis
336,p->10n + 8p,pi-,"p: 0.056, pi-: 0.345","p: 0.014, p: 0.009, p: 0.004, p: 0.013, p: 0.0...",1.200000,res
374,p->n + 2p,,p: 0.122,"p: 0.001, p: 0.075, n: 0.062",1.200000,qel


,reaction,resc = 1,kei,kef,wght_FrInel_N_p1,scatt_lab
32,p + pi+ + pi-->pi- + pi0 + 2n + 2p,,"p: 0.092, pi+: 0.418, pi-: 0.354","p: 0.041, n: 0.082, pi0: 0.414, p: 0.077, pi-:...",1.400000,dis
101,p + pi+ + pi-->pi- + pi0 + n + 3p,,"p: 0.089, pi+: 0.155, pi-: 0.027","p: 0.021, n: 0.087, pi0: 0.13, p: 0.027, pi-: ...",1.400000,dis
262,n + p->9n + 5p,,"n: 0.127, p: 0.304","p: 0.001, p: 0.005, p: 0.011, p: 0.004, n: 0.0...",1.327686,
294,p->2p,pi0,"p: 0.802, pi0: 0.378","p: 0.269, p: 0.526, pi0: 0.378",1.400000,res
379,p + pi+ + pi-->3n + 5p,,"p: 0.191, pi+: 0.434, pi-: 0.097","p: 0.119, p: 0.065, p: 0.136, p: 0.108, p: 0.1...",1.400000,dis
880,p + pi0->pi- + pi+ + 3p,,"p: 0.205, pi0: 0.383","p: 0.086, p: 0.118, p: 0.106, pi+: 0.13, pi-: ...",1.400000,res
1193,p->n + p,pi-,"p: 0.907, pi-: 0.776","p: 0.838, n: 0.061, pi-: 0.776",1.400000,dis
1368,n->2n,p,"n: 0.054, p: 0.321","n: 0.057, n: 0.003, p: 0.321",1.400000,
1452,p + pi-->pi- + n + 2p,,"p: 0.015, pi-: 0.666","p: 0.065, p: 0.03, pi-: 0.161, n: 0.509",1.400000,res
1520,n + p->2n + 2p,,"n: 0.097, p: 0.245","n: 0.084, n: 0.023, p: 0.19, p: 0.062",1.960000,


,reaction,resc = 1,kei,kef,wght_FrPiProd_N_p1,scatt_lab
1015,n + pi+->pi- + pi+ + n + 2p,,"n: 0.857, pi+: 0.25","n: 0.516, p: 0.128, pi-: 0.104, pi+: 0.22, p: ...",1.2,res
1145,p + pi-->pi+ + 4n + 4p,,"p: 0.618, pi-: 0.463","p: 0.364, n: 0.046, pi+: 0.093, p: 0.077, p: 0...",1.2,res
1555,p->pi- + 2p,pi-,"p: 1.409, pi-: 0.292","p: 0.447, p: 0.562, pi-: 0.386, pi-: 0.292",1.2,dis
2038,n + pi+ + pi+->pi- + pi0 + 2n + 7p,pi-,"n: 1.023, pi+: 0.3, pi+: 0.264, pi-: 0.064","n: 0.35, p: 0.46, pi-: 0.107, pi0: 0.247, p: 0...",1.2,dis
2456,p->pi+ + n + p,pi0,"p: 0.682, pi0: 0.231","p: 0.083, n: 0.421, pi+: 0.063, pi0: 0.231",1.2,res
2504,n + pi0->pi- + pi0 + n + 2p,,"n: 0.567, pi0: 0.2","n: 0.133, p: 0.112, pi-: 0.198, pi0: 0.099, p:...",1.2,res
3287,n->pi- + n + p,pi+,"n: 0.843, pi+: 0.331","n: 0.291, p: 0.245, pi-: 0.186, pi+: 0.331",1.2,res
3321,n + pi+->pi- + pi0 + pi+ + n + 2p,pi0,"n: 0.51, pi+: 0.56, pi0: 1.593","n: 0.202, p: 0.116, pi-: 0.071, p: 0.266, pi+:...",1.2,dis
3453,p + pi-->2pi0 + 2n + p,,"p: 0.523, pi-: 0.213","p: 0.295, n: 0.059, pi0: 0.05, pi0: 0.197, n: ...",1.2,dis
3981,p + pi-->pi- + pi0 + n + 2p,,"p: 0.943, pi-: 0.167","p: 0.5, p: 0.297, pi-: 0.03, pi0: 0.135, n: 0.024",1.2,res


,reaction,resc = 1,kei,kef,wght_FrCEx_pi_p1,scatt_lab
62,p + pi- + pi-->pi- + pi0 + pi+ + 3n + p,"pi+, pi+","p: 0.735, pi+: 0.027, pi-: 0.255, pi+: 0.704, ...","n: 0.472, p: 0.251, pi+: 0.027, pi0: 0.214, n:...",1.500000,dis
87,pi+ + pi0->2pi0 + n + p,n,"n: 0.329, pi+: 0.248, pi0: 0.124","n: 0.329, pi0: 0.047, p: 0.199, pi0: 0.101, n:...",1.076411,dis
101,p + pi+ + pi-->pi- + pi0 + n + 3p,,"p: 0.089, pi+: 0.155, pi-: 0.027","p: 0.021, n: 0.087, pi0: 0.13, p: 0.027, pi-: ...",1.439262,dis
274,p + pi+ + pi-->pi0 + 9n + 7p,,"p: 0.131, pi+: 0.021, pi-: 0.452","p: 0.006, p: 0.065, n: 0.061, n: 0.019, pi0: 0...",1.500000,dis
369,pi+ + pi0 + pi+ + pi-->pi- + pi0 + 2pi+ + 2n + 3p,n,"n: 0.253, pi+: 0.471, pi0: 0.021, pi+: 0.093, ...","n: 0.253, p: 0.051, pi+: 0.069, pi-: 0.23, n: ...",1.141557,dis
373,pi0 -> pi+ + n,p,"p: 0.094, pi0: 0.188","p: 0.094, pi+: 0.077, n: 0.098",1.500000,res
385,n + pi+ + pi- + pi+ + pi- + pi+ + pi-->3pi0 + ...,pi+,"n: 0.454, pi+: 0.033, pi+: 0.206, pi-: 0.133, ...","p: 0.037, n: 0.405, pi+: 0.033, n: 0.164, p: 0...",1.678382,dis
435,pi- -> pi0 + n,p,"p: 0.766, pi-: 0.241","p: 0.766, pi0: 0.173, n: 0.056",1.500000,res
542,pi+ -> pi0 + p,"n, pi-","n: 0.19, pi+: 0.111, pi-: 0.674","n: 0.19, pi0: 0.053, p: 0.066, pi-: 0.674",1.500000,dis
584,"pi+ -> pi0 + p, pi0 -> pi0 + pi+","n, pi0","n: 0.734, pi+: 0.139, pi0: 0.333, pi0: 0.221","n: 0.734, pi0: 0.037, p: 0.105, pi0: 0.333, pi...",2.250000,dis


,reaction,resc = 1,kei,kef,wght_FrAbs_pi_p1,scatt_lab
40,pi0->2p,p,"p: 0.44, pi0: 0.09","p: 0.44, p: 0.059, p: 0.118",1.300000,res
70,p + pi+->pi- + 4n + 4p,,"p: 0.049, pi+: 0.219","pi-: 0.34, n: 0.012, p: 0.045, p: 0.049, p: 0....",1.300000,res
77,pi+->6n + 2p,"n, pi0","n: 0.203, pi+: 0.482, pi0: 0.685","n: 0.203, p: 0.05, p: 0.066, n: 0.056, n: 0.07...",1.300000,dis
92,pi+->n + p,"p, pi-","p: 0.084, pi+: 0.08, pi-: 0.052","p: 0.084, n: 0.085, p: 0.107, pi-: 0.052",1.300000,dis
162,p + pi-->6n + 6p,,"p: 0.426, pi-: 0.415","p: 0.091, p: 0.062, p: 0.045, p: 0.062, n: 0.1...",1.300000,res
214,n + pi0->5n + 4p,,"n: 0.111, pi0: 0.177","p: 0.023, n: 0.017, n: 0.025, n: 0.066, p: 0.0...",1.300000,res
227,"Lambda -> K+ + n + Lambda, pi- -> 2n",K+,"Lambda: 0.857, K+: 0.444, pi-: 0.109","Lambda: 0.857, K+: 0.444, n: 0.166, n: 0.077",1.300000,dis
379,p + pi+ + pi-->3n + 5p,,"p: 0.191, pi+: 0.434, pi-: 0.097","p: 0.119, p: 0.065, p: 0.136, p: 0.108, p: 0.1...",1.690000,dis
382,pi-->2n,"p, pi+","p: 0.175, pi+: 0.323, pi-: 0.158","p: 0.175, pi+: 0.323, n: 0.185, n: 0.119",1.300000,dis
512,pi+ + pi-->pi- + pi+ + 6n + 5p,n,"n: 0.447, pi+: 0.554, pi-: 0.48","n: 0.447, p: 0.09, pi+: 0.1, pi-: 0.231, p: 0....",1.173431,dis


,reaction,resc = 1,kei,kef,wght_FrInel_pi_p1,scatt_lab
18,pi0->pi0 + p,n,"n: 0.26, pi0: 0.163","n: 0.26, pi0: 0.121, p: 0.048",1.400000,dis
32,p + pi+ + pi-->pi- + pi0 + 2n + 2p,,"p: 0.092, pi+: 0.418, pi-: 0.354","p: 0.041, n: 0.082, pi0: 0.414, p: 0.077, pi-:...",1.180800,dis
87,pi+ + pi0->2pi0 + n + p,n,"n: 0.329, pi+: 0.248, pi0: 0.124","n: 0.329, pi0: 0.047, p: 0.199, pi0: 0.101, n:...",1.239634,dis
89,pi+->pi+ + p,"p, pi-, pi-","p: 0.566, pi-: 0.045, pi+: 0.267, pi-: 0.282","p: 0.566, pi-: 0.045, pi+: 0.247, p: 0.018, pi...",1.400000,dis
101,p + pi+ + pi-->pi- + pi0 + n + 3p,,"p: 0.089, pi+: 0.155, pi-: 0.027","p: 0.021, n: 0.087, pi0: 0.13, p: 0.027, pi-: ...",1.222143,dis
159,p + pi-->pi- + n + 2p,,"p: 0.351, pi-: 0.116","n: 0.22, p: 0.137, pi-: 0.081, p: 0.025",1.400000,dis
279,pi0->pi0 + p,p,"p: 0.507, pi0: 0.098","p: 0.507, pi0: 0.054, p: 0.043",1.400000,res
369,pi+ + pi0 + pi+ + pi-->pi- + pi0 + 2pi+ + 2n + 3p,n,"n: 0.253, pi+: 0.471, pi0: 0.021, pi+: 0.093, ...","n: 0.253, p: 0.051, pi+: 0.069, pi-: 0.23, n: ...",1.035957,dis
448,pi+->pi+ + n,"n, pi0","n: 0.097, pi+: 0.179, pi0: 1.41","n: 0.097, pi+: 0.071, n: 0.093, pi0: 1.41",1.400000,dis
456,n + pi0->pi0 + n + 2p,,"n: 0.036, pi0: 0.165","p: 0.052, n: 0.029, pi0: 0.164, p: 0.004",1.400000,res


,reaction,resc = 1,kei,kef,wght_FrPiProd_pi_p1,scatt_lab
24,pi0->pi- + pi+ + n,p,"p: 0.971, pi0: 0.499","p: 0.971, n: 0.226, pi+: 0.037, pi-: 0.095",1.2,res
62,p + pi- + pi-->pi- + pi0 + pi+ + 3n + p,"pi+, pi+","p: 0.735, pi+: 0.027, pi-: 0.255, pi+: 0.704, ...","n: 0.472, p: 0.251, pi+: 0.027, pi0: 0.214, n:...",1.2,dis
369,pi+ + pi0 + pi+ + pi-->pi- + pi0 + 2pi+ + 2n + 3p,n,"n: 0.253, pi+: 0.471, pi0: 0.021, pi+: 0.093, ...","n: 0.253, p: 0.051, pi+: 0.069, pi-: 0.23, n: ...",1.2,dis
740,pi+ + pi0->pi0 + pi+ + 4n + 3p,"n, pi0, pi0, pi0","n: 0.173, pi+: 0.188, pi0: 0.409, pi0: 0.197, ...","n: 0.173, p: 0.015, p: 0.024, p: 0.048, n: 0.0...",1.2,dis
880,p + pi0->pi- + pi+ + 3p,,"p: 0.205, pi0: 0.383","p: 0.086, p: 0.118, p: 0.106, pi+: 0.13, pi-: ...",1.2,res
887,n + pi+ + pi0->pi- + pi+ + 3n + 2p,,"n: 0.266, pi+: 0.046, pi0: 0.963","p: 0.07, n: 0.21, n: 0.162, p: 0.028, n: 0.358...",1.2,dis
895,pi+->2pi0 + p,"p, pi-, pi-","p: 0.178, pi-: 0.804, pi+: 0.464, pi-: 0.695","p: 0.178, pi-: 0.804, p: 0.036, pi0: 0.104, pi...",1.2,dis
947,n + pi+ + pi0->pi- + 2pi+ + 10n + 10p,,"n: 0.163, pi+: 0.009, pi0: 0.715","p: 0.009, p: 0.012, p: 0.019, p: 0.017, p: 0.0...",1.2,dis
998,n + pi-->pi- + pi0 + n + 2p,"pi0, pi+","n: 0.311, pi0: 0.028, pi+: 0.073, pi-: 0.839","p: 0.117, n: 0.188, pi0: 0.028, pi+: 0.073, p:...",1.2,dis
1017,p + pi+ + pi-->2pi0 + pi+ + 4n + 5p,,"p: 0.647, pi+: 0.579, pi-: 0.201","p: 0.103, p: 0.28, p: 0.055, p: 0.122, n: 0.06...",1.2,dis
